## UNET

**U-Net: Convolutional Networks for Biomedical Image Segmentation** [article](https://arxiv.org/pdf/1505.04597) (2015)

U-Net is a convolutional neural network (CNN) designed for semantic segmentation tasks. Its name comes from its U-shaped architecture, which consists of an encoder followed by a decoder path. This unique structure allows U-Net to capture context at multiple scales while preserving spatial information.

In the introduction of the article, it states:

> In this paper, we build upon a novel and elegant architecture, the so-called "fully convolutional network". **We modify and extend this architecture to work with very few training images and to yield more precise segmentations**. The main idea is to supplement a usual contracting network by successive layers, where pooling operators are replaced by upsampling operators that increase the resolution of the output. In order to localize high-resolution features, the upsampling path combines feature maps from the encoder.  
> A key modification in our architecture is that the upsampling path also has a large number of feature channels, which allows the network to propagate context information to higher resolution layers. As a result, the expansive path is more or less symmetric to the contracting path, yielding a U-shaped architecture. The network has no fully connected layers and only uses valid parts of each convolution, i.e., the segmentation map contains only those pixels for which the full context is available in the input image.  
> This strategy enables seamless segmentation of arbitrarily large images by using an overlap-tile strategy. To predict the pixels at the border of the image, the missing context is extrapolated by mirroring the input image. This tiling strategy is important for applying the network to large images, since otherwise the resolution would be limited by the GPU memory.

This network performs well even with small datasets, which is common in medical environments. Although it is lighter than other segmentation architectures, it still achieves high performance, allowing efficient deployment on systems with limited computational resources.

### Architecture

The architecture is divided into two main parts:

#### Encoder

The encoder architecture it is essentially a CNN following the design principles of VGG networks.

From the article:

> It follows the typical architecture of a convolutional network. It consists of the repeated application of two 3×3 convolutions (unpadded), each followed by a ReLU and a 2×2 max pooling operation with stride 2 for downsampling. At each downsampling step, we double the number of feature channels.

#### Decoder

The decoder introduces a new operation, **up-convolution**, which acts as the opposite of the encoder’s max pooling. It increases the size of the feature map and is followed by a convolution to reduce the number of channels by half.

In PyTorch, this operation is implemented with a single layer called [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html). There’s also a [visual explanation here](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md).

From the article:

> Every step in the expansive path consists of an upsampling of the feature map followed by a 2×2 convolution (“up-convolution”) that halves the number of feature channels. Then, a concatenation with the correspondingly cropped feature map from the contracting path is performed, and two 3×3 convolutions, each followed by a ReLU, are applied. In the final layer, a 1×1 convolution is used to map each 64-component feature vector to the desired number of classes.

![UNet](../assets/bloc2/unet.png "UNet Architecture")

In total, the network has **23 convolutional layers**. To allow seamless tiling of the output segmentation map, it’s important to choose the input tile size such that all 2×2 max pooling operations are applied on feature maps with even width and height. Due to unpadded convolutions, the output image is smaller than the input.

![mosaic](../assets/bloc2/mosaic.png "UNet Tiling Strategy")

The output layer uses a **softmax activation function**, since the dataset contained multiple types of cells to segment.
